Name:Mahyar Jahani nasab


# Bootstrap

# Install

In [1]:
! pip install -U torch datasets pyperclip icecream numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Download the Data

In [2]:
!wget https://files.lilf.ir/Black%20Luminary.txt

--2023-04-29 09:42:30--  https://files.lilf.ir/Black%20Luminary.txt
Resolving files.lilf.ir (files.lilf.ir)... 82.102.11.148
Connecting to files.lilf.ir (files.lilf.ir)|82.102.11.148|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3148450 (3.0M) [text/plain]
Saving to: ‘Black Luminary.txt.3’

Black Luminary.txt. 100%[===================>]   3.00M  4.02MB/s    in 0.7s    

2023-04-29 09:42:32 (4.02 MB/s) - ‘Black Luminary.txt.3’ saved [3148450/3148450]



In [3]:
! ls -lh

total 13M
-rw-r--r-- 1 root root 3.1M Oct 14  2021 'Black Luminary.txt'
-rw-r--r-- 1 root root 3.1M Oct 14  2021 'Black Luminary.txt.1'
-rw-r--r-- 1 root root 3.1M Oct 14  2021 'Black Luminary.txt.2'
-rw-r--r-- 1 root root 3.1M Oct 14  2021 'Black Luminary.txt.3'
drwxr-xr-x 1 root root 4.0K Apr 27 13:35  sample_data


In [4]:
! realpath *.txt

/content/Black Luminary.txt


# User Config

In [5]:
data_paths = [
    '/content/Black Luminary.txt',
    ]

## imports

In [6]:
import pyperclip

In [7]:
from icecream import ic

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
device = torch.device("cpu")
#: We will set device again in the training loop.

In [10]:
import datasets as D

In [11]:
import numpy
np = numpy

import statistics

# Utils

In [12]:
class NumpyPrintOptions:
    def __init__(self, **kwargs):
        self.options = kwargs
        self.original_options = np.get_printoptions()

    def __enter__(self):
        np.set_printoptions(**self.options)

    def __exit__(self, exc_type, exc_value, traceback):
        np.set_printoptions(**self.original_options)

class NoTruncationNumpyPrintOptions(NumpyPrintOptions):
    def __init__(self):
        super().__init__(
            threshold=np.inf, 
            linewidth=200, 
            suppress=True, 
            precision=4
        )

In [13]:
import jax

def torch_shape_get(input):
    def h_shape_get(x):
        return x.dtype, x.shape

    return jax.tree_map(h_shape_get, input)

In [14]:
def has_nan(tensor):
    return torch.any(torch.isnan(tensor))

In [15]:
class ModelEvalMode:
    def __init__(self, model):
        self.model = model

    def __enter__(self):
        self.model.eval()

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.model.train()

# Data

In [16]:
d = D.load_dataset("text",
                         data_files=data_paths, sample_by="paragraph")
d

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 18423
    })
})

In [17]:
d = d['train']
d

Dataset({
    features: ['text'],
    num_rows: 18423
})

In [18]:
d[1000:1010]

{'text': ['Professor Snape threw him backwards, and Harry stumbled, but just managed to keep standing.',
  "'I understand, sir. I apologise if my careless words have offended.'",
  "'Get going then!' The man turned around and marched towards his desk.",
  'Not keen on his company for the moment, Harry hastily made his way towards the hall.',
  "Merlin, did he have to grab my arm like that? If being implicated in murder is child's play to him, then I can indeed do without partaking in his 'problems'…",
  '~BLHD~',
  "Harry paused before entering the Great Hall and forced his countenance as hard as he could into a blank expression. He must not let up; he must not relent for one second. Weakness would not help him here. It might also be prudent to distance himself a bit from other people for a while to limit the damage to their reputation and family; depending on how the whole situation turned out, the political fallout could be immense. With a sense of foreboding, he imagined Daphne's re

In [19]:
def str_to_np(s, dtype=np.int8):
    s = s.encode('ascii', errors='ignore')
    return np.frombuffer(s, dtype=dtype)

str_to_np('hello')

array([104, 101, 108, 108, 111], dtype=int8)

In [20]:
def str_to_onehot(s):
    return np.eye(256)[str_to_np(s)]

In [21]:
dc = d.map(lambda batch: {'input': [str_to_np(t).astype(np.int32) + 1 for t in batch['text']]}, batched=True) #: added one to the char indices to make zero available for the pad token
dc

Dataset({
    features: ['text', 'input'],
    num_rows: 18423
})

In [22]:
dc = dc.filter(lambda x: (len(x['input']) > 30 and len(x['text'].split()) > 4), batched=False)
dc

Dataset({
    features: ['text', 'input'],
    num_rows: 16371
})

In [23]:
dc.set_format("torch", columns=["input",])

In [24]:
torch_shape_get(dc[1000:1010]['input'])

[(torch.int64, torch.Size([1126])),
 (torch.int64, torch.Size([727])),
 (torch.int64, torch.Size([163])),
 (torch.int64, torch.Size([232])),
 (torch.int64, torch.Size([106])),
 (torch.int64, torch.Size([88])),
 (torch.int64, torch.Size([82])),
 (torch.int64, torch.Size([69])),
 (torch.int64, torch.Size([127])),
 (torch.int64, torch.Size([64]))]

In [25]:
dc = dc.shuffle()

In [26]:
dcs = dc.train_test_split(test_size=0.2)
dcs

DatasetDict({
    train: Dataset({
        features: ['text', 'input'],
        num_rows: 13096
    })
    test: Dataset({
        features: ['text', 'input'],
        num_rows: 3275
    })
})

# Model

In [27]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        
        self.embedding = nn.Embedding(256, 64)
        self.gru = nn.GRU(64, 128, 2, batch_first=True, dropout=0.1)
        self.fc1 = nn.Linear(128, 256)


    def forward(self, x, hidden=None):
        # x: list of tensors shaped (seq_length, batch_size)
        # The seq_length will not necessarily be equal for all list items!
        # 
        # Convert the list of tensors to a padded tensor
        x_padded = nn.utils.rnn.pad_sequence(x, batch_first=True, padding_value=0)

        # Pass the padded tensor through the embedding layer
        embedded = self.embedding(x_padded)

        # Pass the embedded tensor through the GRU layer
        output, hidden = self.gru(embedded, hidden)

        # Pass the last hidden state through the linear layer
        output = self.fc1(output)

        return output,hidden

In [28]:
def loss_fn(y, y_hat):
    # Subtract 1 from the token indices to convert them to ASCII indices
    y = [t - 1 for t in y]
    y_padded = torch.nn.utils.rnn.pad_sequence(y, batch_first=True, padding_value=-1)
    y_hat_flat = y_hat.view(-1, y_hat.size(-1))
    y_padded_flat = y_padded.view(-1)
    mask = (y_padded_flat != -1)
    loss = torch.nn.functional.cross_entropy(y_hat_flat[mask], y_padded_flat[mask])
    average_loss = loss.mean()
    return average_loss



In [29]:
def shift_left(tensor_list, pad_value=0.0):
    shifted_tensors = []
    for tensor in tensor_list:
        # Shift tensor one position to the left
        shifted = torch.cat([tensor[1:], tensor.new([int(pad_value)])], dim=0)
        shifted_tensors.append(shifted)
    
    return shifted_tensors

# Example usage:
input_ids = [torch.tensor([1, 2, 3, 4]), torch.tensor([5, 6, 7, 8])]
print("Input Ids:")
print(input_ids)

target_ids = shift_left(input_ids)
print("Shifted Left (Target Ids):")
print(target_ids)

Input Ids:
[tensor([1, 2, 3, 4]), tensor([5, 6, 7, 8])]
Shifted Left (Target Ids):
[tensor([2, 3, 4, 0]), tensor([6, 7, 8, 0])]


# Beam Search Generation

In [30]:
import torch
import heapq

def tensor_to_string(tensor):
    chars = [chr(c) for c in tensor]
    return ''.join(chars)

def tensor_append_scalar(tensor, scalar):
    scalar_tensor = torch.tensor(scalar).view(1)  # Add a dimension to match the original tensor's dimensions
    scalar_tensor = scalar_tensor.to(device)

    # Append the scalar to the original tensor
    result = torch.cat((tensor, scalar_tensor), dim=0)
    return result


def generate_next_top_k(model, input_sequence, k):
    logits, _ = model.forward([input_sequence])
    logits = logits[0, -1, :]
    # ic(torch_shape_get(logits))
    
    probabilities = torch.softmax(logits, dim=-1)
    # ic(torch_shape_get(probabilities))

    top_k_values, top_k_indices = torch.topk(probabilities, k)

    return [(tensor_append_scalar(input_sequence, idx.item() + 1), log_prob.item()) for idx, log_prob in zip(top_k_indices, top_k_values.log())]

def beam_search(model, desired_length, starting_string, k=5):
    with ModelEvalMode(model), torch.no_grad():
      input_sequence = torch.tensor(str_to_np(starting_string).astype(np.int32) + 1, dtype=torch.long)
      input_sequence = input_sequence.to(device)
      # ic(torch_shape_get(input_sequence))
      
      log_prob = 0.0

      beam = [(input_sequence, log_prob)]

      while len(beam[0][0]) < desired_length:
          new_beam = []
          for seq, log_prob in beam:
              next_top_k = generate_next_top_k(model, seq, k)
              new_beam.extend([(new_seq, new_log_prob + log_prob) for new_seq, new_log_prob in next_top_k])

          beam = heapq.nlargest(k, new_beam, key=lambda x: x[1])

      return [tensor_to_string(seq - 1) for seq, _ in beam]

In [31]:
a = str_to_np("Harry ").astype(np.int32) + 1
ic(a)
tensor_to_string(a - 1)

ic| a: array([ 73,  98, 115, 115, 122,  33], dtype=int32)


'Harry '

In [32]:
def eval_gen(*args, display=999999, **kwargs):
    generated_texts = beam_search(
        *args, **kwargs,
    )

    for idx, text in enumerate(generated_texts):
        if idx >= display:
            break
        
        print(f"Generated text {idx + 1}: {text}")

# Train

In [33]:
dt = dcs['train']
dt

Dataset({
    features: ['text', 'input'],
    num_rows: 13096
})

In [34]:
if torch.cuda.is_available():
	device = 'cuda'
	non_blocking = True
elif True:
	device = 'cpu'
	non_blocking = False
else:
	#: causes NaNs
	device = 'mps' 
	non_blocking = False

i = 0

#: Feel free to edit these hyperparameters or the optimizer
#: You might want to use a learning-rate scheduler, such as
#: [ReduceLROnPlateau --- PyTorch 2.0 documentation](https://pytorch.org/docs/stable/generated/torch.optim.lr_scheduler.ReduceLROnPlateau.html)
epochs = 600
batch_size = 4096
learning_rate = 0.0007
max_len = 0
m = Model().to(device=device, non_blocking=non_blocking)
m.train()

optimizer = torch.optim.AdamW(m.parameters(), lr=learning_rate)
#from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.2,  min_lr=0.0001)

counter = 0
for epoch in range(epochs):
	dt = dt.shuffle()
	for i in range(0, len(dt), batch_size):
		batch = dt[i:i+batch_size]
		inputs = batch['input']

		if max_len > 0:
			inputs = list(map(lambda x: x[:max_len] if len(x) > max_len else x, inputs))

		lens = [len(seq) for seq in inputs]
		current_max_len = max(lens)
		mean_len = statistics.mean(lens)
		# ic(current_max_len, mean_len)

		inputs = list(map(lambda x: x.to(device, non_blocking=non_blocking), inputs))
		# ic(torch_shape_get(inputs))

		targets = shift_left(inputs)

		# truncate the inputs and targets to a fixed length for backpropagation
		trunc_len = 100 
		inputs_trunc = [seq[-trunc_len:] for seq in inputs]
		targets_trunc = [seq[-trunc_len:] for seq in targets]


		# compute the outputs and loss
		outputs_padded, _ = m(inputs_trunc)

		average_loss= loss_fn(targets_trunc, outputs_padded)
	
		l = average_loss / mean_len
		# update the parameters
		m.zero_grad()
		average_loss.backward()
		torch.nn.utils.clip_grad_norm_(m.parameters(), 1.0)
		optimizer.step()
		if counter % 1 == 0:
			l = l.item()
			print(f"loss: {l:>7f}  [{counter:>5d}, epoch={epoch}]")
		
		counter += 1

	# print(f"loss: {l:>7f}  [{counter-1:>5d}, epoch={epoch} finished!]")
	if epoch % 15 == 0:
		eval_gen(display=3, model=m, desired_length=100, starting_string="Harry ", k=32)


loss: 0.029138  [    0, epoch=0]
loss: 0.029253  [    1, epoch=0]
loss: 0.029779  [    2, epoch=0]
loss: 0.029337  [    3, epoch=0]
Generated text 1: Harry   i  i   i   i  i   i   i  i   i   i   i   i  i   i  i   i  i   i  i   i   i   i   i   i   i  
Generated text 2: Harry   i  i   i   i  i   i   i  i   i   i   i   i   i   i   i   i   i   i   i   i   i   i   i   i  
Generated text 3: Harry   i  i   i   i   i   i   i   i   i   i  i   i  i   i   i   i   i   i   i   i   i   i   i   i  
loss: 0.028438  [    4, epoch=1]
loss: 0.028410  [    5, epoch=1]
loss: 0.028104  [    6, epoch=1]
loss: 0.026729  [    7, epoch=1]
loss: 0.026818  [    8, epoch=2]
loss: 0.025806  [    9, epoch=2]
loss: 0.025696  [   10, epoch=2]
loss: 0.023698  [   11, epoch=2]
loss: 0.023109  [   12, epoch=3]
loss: 0.022427  [   13, epoch=3]
loss: 0.021165  [   14, epoch=3]
loss: 0.020700  [   15, epoch=3]
loss: 0.019809  [   16, epoch=4]
loss: 0.019162  [   17, epoch=4]
loss: 0.018805  [   18, epoch=4]
loss: 0.019243  

In [35]:
eval_gen(display=3, model=m, desired_length=100, starting_string="Harry ", k=32)

Generated text 1: Harry couldn't have something something with the first time. 'What?' asked Hermione. 'What?' asked H
Generated text 2: Harry couldn't have something something like the first time. 'What?' asked Hermione. 'What?' asked H
Generated text 3: Harry couldn't have something something with the first time. 'What?' asked Hermione. 'What?' asked D


In [36]:
eval_gen(display=50, model=m, desired_length=250, starting_string="Harry ", k=100)

Generated text 1: Harry nodded towards the door. 'What do you think you know?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. '
Generated text 2: Harry nodded towards the door. 'What do you think you know?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. '
Generated text 3: Harry nodded towards the door. 'What do you think you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. '
Generated text 4: Harry nodded towards the door. 'What do you think you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione

In [37]:
eval_gen(display=50, model=m, desired_length=250, starting_string="Arcturus ", k=100)

Generated text 1: Arcturus and Hermione and Harry with his eyes. 'What is it?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. '
Generated text 2: Arcturus and Hermione and Harry with a grin. 'What do you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. '
Generated text 3: Arcturus and Hermione and Harry with a grin. 'What do you mean?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. '
Generated text 4: Arcturus and Hermione and Harry with his eyes. 'What is it?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione

In [38]:
eval_gen(display=50, model=m, desired_length=150, starting_string="Draco ", k=100)

Generated text 1: Draco expression of his eyes. 'What do you mean?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' aske
Generated text 2: Draco expression of his eyes. 'What do you mean?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'You're 
Generated text 3: Draco expression of his eyes. 'What do you mean?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'You're 
Generated text 4: Draco expression of his eyes. 'What do you mean?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'You're 
Generated text 5: Draco expression of his eyes. 'What do you mean?' asked Hermione. 'What are you?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'You
Generated text 6: Draco expression of his eyes. 'What do you mean?' asked Hermione. 'What is it?' asked Hermione. 'What?' asked Hermione. 'What?' asked Her

In [39]:
eval_gen(display=50, model=m, desired_length=150, starting_string="Harry looked at ", k=100)

Generated text 1: Harry looked at Harry's eyes. 'What do you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' aske
Generated text 2: Harry looked at her eyes. 'What do you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Ha
Generated text 3: Harry looked at her eyes. 'What do you know?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' aske
Generated text 4: Harry looked at her eyes. 'What do you know?' asked Hermione. 'What?' asked Hermione. 'You know?' asked Hermione. 'What?' asked Hermione. 'What?' aske
Generated text 5: Harry looked at her eyes. 'What do you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'You know?' aske
Generated text 6: Harry looked at her eyes. 'What do you know?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'What?' asked Hermione. 'Wh